In [4]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
from datetime import date
import urllib.request
import time
from bs4 import BeautifulSoup
import streamlit
import matplotlib.pyplot as plt

## DOWNLOADING DATA FROM YAHOO FINANCE

In [5]:
#import yfinance - be able to download data from yahoo finance
import yfinance as yf

In [6]:
#set the beginning of time series to 1st of Jan 2010
BEGINNING = "2010-01-01"

In [7]:
#Set today's date
TODAY = date.today()
print(TODAY)

2022-01-25


In [8]:
#Date is in righ format - but to ensure it will be also in the figh format in the future - state it in the code
TODAY = date.today().strftime("%Y-%m-%d")

In [9]:
#Get stock names S&P 500 from Wikipedia
req = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(req.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})

In [10]:
#prepare variables 
tickers=[]
sector = []
for row in table.findAll('tr')[1:]:
        tic = row.findAll('td')[0].text
        #sector
        sec = row.findAll('td')[4].text
        tickers.append(tic)
        sector.append(sec)

In [11]:
print(tickers)

['MMM\n', 'AOS\n', 'ABT\n', 'ABBV\n', 'ABMD\n', 'ACN\n', 'ATVI\n', 'ADBE\n', 'AAP\n', 'AES\n', 'AFL\n', 'A\n', 'AIG\n', 'APD\n', 'AKAM\n', 'ALK\n', 'ALB\n', 'ARE\n', 'ALGN\n', 'ALLE\n', 'LNT\n', 'ALL\n', 'GOOGL\n', 'GOOG\n', 'MO\n', 'AMZN\n', 'AMCR\n', 'AMD\n', 'AEE\n', 'AAL\n', 'AEP\n', 'AXP\n', 'AMT\n', 'AWK\n', 'AMP\n', 'ABC\n', 'AME\n', 'AMGN\n', 'APH\n', 'ADI\n', 'ANSS\n', 'ANTM\n', 'AON\n', 'APA\n', 'AAPL\n', 'AMAT\n', 'APTV\n', 'ADM\n', 'ANET\n', 'AJG\n', 'AIZ\n', 'T\n', 'ATO\n', 'ADSK\n', 'ADP\n', 'AZO\n', 'AVB\n', 'AVY\n', 'BKR\n', 'BLL\n', 'BAC\n', 'BBWI\n', 'BAX\n', 'BDX\n', 'BRK.B\n', 'BBY\n', 'BIO\n', 'TECH\n', 'BIIB\n', 'BLK\n', 'BK\n', 'BA\n', 'BKNG\n', 'BWA\n', 'BXP\n', 'BSX\n', 'BMY\n', 'AVGO\n', 'BR\n', 'BRO\n', 'BF.B\n', 'CHRW\n', 'CDNS\n', 'CZR\n', 'CPB\n', 'COF\n', 'CAH\n', 'KMX\n', 'CCL\n', 'CARR\n', 'CTLT\n', 'CAT\n', 'CBOE\n', 'CBRE\n', 'CDW\n', 'CE\n', 'CNC\n', 'CNP\n', 'CDAY\n', 'CERN\n', 'CF\n', 'CRL\n', 'SCHW\n', 'CHTR\n', 'CVX\n', 'CMG\n', 'CB\n', 'CHD\n', 

In [12]:
tickers = list(map(lambda s: s.strip(), tickers))

In [15]:
industries = list(map(lambda s: s.strip(), sector))

In [16]:
tickerdf = pd.DataFrame(tickers,columns=['ticker'])
sectordf = pd.DataFrame(industries,columns=['industry'])

In [17]:
df = pd.concat([tickerdf, sectordf], axis=1)
print(df)

    ticker                            industry
0      MMM            Industrial Conglomerates
1      AOS                   Building Products
2      ABT               Health Care Equipment
3     ABBV                     Pharmaceuticals
4     ABMD               Health Care Equipment
..     ...                                 ...
500    YUM                         Restaurants
501   ZBRA  Electronic Equipment & Instruments
502    ZBH               Health Care Equipment
503   ZION                      Regional Banks
504    ZTS                     Pharmaceuticals

[505 rows x 2 columns]


In [18]:
df2 = df.reindex(tickerdf.index)

In [19]:
df2

,ticker,industry
0,MMM,Industrial Conglomerates
1,AOS,Building Products
2,ABT,Health Care Equipment
3,ABBV,Pharmaceuticals
4,ABMD,Health Care Equipment
...,...,...
500,YUM,Restaurants
501,ZBRA,Electronic Equipment & Instruments
502,ZBH,Health Care Equipment
503,ZION,Regional Banks


In [20]:
#ALL TICKERS FROM S&P
tick = df['ticker'].to_numpy()
tick

array(['MMM', 'AOS', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADBE', 'AAP',
       'AES', 'AFL', 'A', 'AIG', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE',
       'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN',
       'AMCR', 'AMD', 'AEE', 'AAL', 'AEP', 'AXP', 'AMT', 'AWK', 'AMP',
       'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'ANTM', 'AON', 'APA',
       'AAPL', 'AMAT', 'APTV', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO',
       'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'BKR', 'BLL', 'BAC', 'BBWI',
       'BAX', 'BDX', 'BRK.B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BK',
       'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO',
       'BF.B', 'CHRW', 'CDNS', 'CZR', 'CPB', 'COF', 'CAH', 'KMX', 'CCL',
       'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP',
       'CDAY', 'CERN', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB',
       'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CTXS', 'CLX',
       'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP'

In [21]:
tick_to_download = tick.tolist()
tick_to_download

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ABMD',
 'ACN',
 'ATVI',
 'ADBE',
 'AAP',
 'AES',
 'AFL',
 'A',
 'AIG',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AMD',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'ANTM',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'BKR',
 'BLL',
 'BAC',
 'BBWI',
 'BAX',
 'BDX',
 'BRK.B',
 'BBY',
 'BIO',
 'TECH',
 'BIIB',
 'BLK',
 'BK',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'CHRW',
 'CDNS',
 'CZR',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'CNC',
 'CNP',
 'CDAY',
 'CERN',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CTXS',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMC

In [22]:
#SUBSET
tick2=tick_to_download[:3].copy()
tick2

['MMM', 'AOS', 'ABT']

In [23]:
tick_to_download

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ABMD',
 'ACN',
 'ATVI',
 'ADBE',
 'AAP',
 'AES',
 'AFL',
 'A',
 'AIG',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AMD',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'ANTM',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'BKR',
 'BLL',
 'BAC',
 'BBWI',
 'BAX',
 'BDX',
 'BRK.B',
 'BBY',
 'BIO',
 'TECH',
 'BIIB',
 'BLK',
 'BK',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'CHRW',
 'CDNS',
 'CZR',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'CNC',
 'CNP',
 'CDAY',
 'CERN',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CTXS',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMC

In [91]:
print(type(tick_to_download))
tick_to_download

<class 'list'>


['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ABMD',
 'ACN',
 'ATVI',
 'ADBE',
 'AAP',
 'AES',
 'AFL',
 'A',
 'AIG',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AMD',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'ANTM',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'BKR',
 'BLL',
 'BAC',
 'BBWI',
 'BAX',
 'BDX',
 'BRK.B',
 'BBY',
 'BIO',
 'TECH',
 'BIIB',
 'BLK',
 'BK',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'CHRW',
 'CDNS',
 'CZR',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'CNC',
 'CNP',
 'CDAY',
 'CERN',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CTXS',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMC

In [57]:
#ALL - NOT RUN
#for tick_to_download in tick_to_download:
#   data = yf.download(tick_to_download,start=BEGINNING,end=TODAY)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [92]:
#ALL - NOT RUN
#for tick in tick_to_download:
#    data = yf.download(tick_to_download,start=BEGINNING,end=TODAY)

[*********************100%***********************]  505 of 505 completed

2 Failed downloads:
- BRK.B: No data found, symbol may be delisted
- BF.B: None
[************          25%                       ]  126 of 505 completed

KeyboardInterrupt: 

[****************      34%                       ]  173 of 505 completed

In [58]:
#CODE ON SUBSET
#for tick2 in tick2:
#    data = yf.download(tick2,start=BEGINNING,end=TODAY)
#    print(tick2,data)

[*********************100%***********************]  1 of 1 completed
MMM                   Open        High         Low       Close   Adj Close  \
Date                                                                     
2009-12-31   83.790001   84.080002   82.540001   82.669998   59.639809   
2010-01-04   83.089996   83.449997   82.669998   83.019997   59.892311   
2010-01-05   82.800003   83.230003   81.699997   82.500000   59.517166   
2010-01-06   83.879997   84.599998   83.510002   83.669998   60.361221   
2010-01-07   83.320000   83.760002   82.120003   83.730003   60.404491   
...                ...         ...         ...         ...         ...   
2022-01-14  180.860001  180.860001  177.669998  178.740005  178.740005   
2022-01-18  178.179993  178.550003  176.169998  178.279999  178.279999   
2022-01-19  179.830002  180.869995  177.630005  178.479996  178.479996   
2022-01-20  178.580002  179.220001  173.309998  173.570007  173.570007   
2022-01-21  173.399994  176.020004  172

In [24]:
#DOWNLOADING DATA
data = yf.download(tick_to_download,start=BEGINNING,end=TODAY)

2022-01-25 17:19:59.450 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


[*********************100%***********************]  505 of 505 completed

2 Failed downloads:
- BRK.B: No data found, symbol may be delisted
- BF.B: None


In [25]:
print(data)

             Adj Close                                                 \
                     A        AAL         AAP        AAPL        ABBV   
Date                                                                    
2009-12-31   20.284777   4.562869   38.527885    6.452592         NaN   
2010-01-04   20.434929   4.496877   38.432693    6.553025         NaN   
2010-01-05   20.212954   5.005957   38.204258    6.564353         NaN   
2010-01-06   20.141142   4.798555   38.537392    6.459941         NaN   
2010-01-07   20.115026   4.939966   38.527885    6.447998         NaN   
...                ...        ...         ...         ...         ...   
2022-01-18  140.470001  17.900000  237.910004  169.800003  136.779999   
2022-01-19  140.429993  17.309999  234.339996  166.229996  135.210007   
2022-01-20  139.479996  16.760000  229.330002  164.509995  133.029999   
2022-01-21  137.509995  16.299999  226.149994  162.410004  131.979996   
2022-01-24  138.119995  16.410000  232.809998  161.

In [26]:
data= data.reset_index()

In [2]:
print(data)

NameError: name 'data' is not defined

In [34]:
data.head()

Date  Adj Close                                                       \
                      A       AAL        AAP      AAPL ABBV        ABC  ABMD   
0 2009-12-31  20.284777  4.562869  38.527885  6.452592  NaN  21.707573  8.73   
1 2010-01-04  20.434929  4.496877  38.432693  6.553025  NaN  22.173862  8.74   
2 2010-01-05  20.212954  5.005957  38.204258  6.564353  NaN  22.015661  8.53   
3 2010-01-06  20.141142  4.798555  38.537392  6.459941  NaN  21.807489  8.40   
4 2010-01-07  20.115026  4.939966  38.527885  6.447998  NaN  21.457771  8.40   

                         ...     Volume                                        \
         ABT        ACN  ...        XEL       XLNX         XOM       XRAY XYL   
0  19.608133  32.524155  ...  1364700.0  2249000.0  18852300.0   489300.0 NaN   
1  19.778828  32.970871  ...  2670400.0  2824700.0  27809100.0  1051400.0 NaN   
2  19.619032  33.174648  ...  4321400.0  3469700.0  30174700.0   763400.0 NaN   
3  19.727978  33.527321  ...  2164500.0  6329200.0  35044700.0  1595100.0 NaN   
4  19.891415  33.495968  ...  3041700.0  9252700.0  27192100.0  1096100.0 NaN   

                                                   
         YUM        ZBH      ZBRA        ZION ZTS  
0  1790217.0   562300.0  184800.0   3239800.0 NaN  
1  2962274.0   782400.0  168800.0   3974600.0 NaN  
2  3298757.0  1718100.0  168800.0   5605500.0 NaN  
3  4178981.0  1277300.0  385300.0  12615200.0 NaN  
4  2452472.0  1683500.0  183600.0  24716800.0 NaN  

[5 rows x 3031 columns]